In [61]:
import os
import pandas as pd

In [62]:
text = """
Gli abitanti di Little Hangleton la chiamavano ancora Casa Riddle, anche se erano passati tanti anni da quando i Riddle ci abitavano. 
Si trovava sulla collina che dominava il villaggio: alcune delle finestre erano inchiodate, al tetto mancavano delle tegole e 
l'edera cresceva incolta sulla facciata. Un tempo Casa Riddle era stata una dimora elegante, certo l'edificio più vasto e grandioso 
nel raggio di chilometri, ma ora era umida, desolata e disabitata. Gli hangletoniani convenivano tutti che la vecchia casa era 
sinistra. Mezzo secolo prima, qualcosa di strano e terribile era successo là dentro, qualcosa di cui gli abitanti più anziani del 
villaggio amavano ancora discutere quando erano a corto di pettegolezzi. La storia era stata ripetuta così tante volte, e vi erano 
stati aggiunti così tanti fronzoli che nessuno era più certo di quale fosse la verità. Ogni versione del racconto, comunque, 
cominciava allo stesso modo: cinquant'anni prima, all'alba di una bella giornata d'estate, quando Casa Riddle era ancora ben tenuta e 
imponente, una cameriera era entrata in salotto e aveva trovato morti tutti e tre i Riddle. La cameriera era corsa urlando giù per la 
collina fino al villaggio e aveva radunato tutte le persone che poteva. Sono là stesi con gli occhi spalancati! Freddi come il 
ghiaccio! Ancora vestiti per la cena! Fu chiamata la polizia e tutta quanta Little Hangleton si crogiolò in una curiosità atterrita e 
in una malcelata eccitazione. Nessuno si sforzò di fingersi addolorato per i Riddle, che erano stati assolutamente impopolari. Gli 
anziani signori Riddle, marito e moglie, erano ricchi, snob e sgarbati, e loro figlio Tom, ormai adulto, era anche peggio. Tutto 
quello che importava agli abitanti era l'identità dell'assassino: chiaramente, tre persone in apparenza sane non morivano di colpo per 
cause naturali nella stessa notte. L'Impiccato, il pub locale, fece affari d'oro quella sera: il villaggio al completo accorse per 
discutere gli omicidi. E la ricompensa per quell'uscita serale arrivò quando la cuoca dei Riddle fece un ingresso teatrale e annunciò 
al pub improvvisamente silenzioso che un uomo chiamato Frank Bryce era stato appena arrestato.
"""

In [63]:
def get_all_csv_files_rec(dir):
    """Ritorna una lista di tutti i file .csv nella directory e nelle sottodirectory."""
    csv_files = []
    for root, dirs, files in os.walk(dir):
        for file in files:
            if file.endswith('.csv'):
                csv_files.append(os.path.join(root, file))
    return csv_files
all_csv_files = get_all_csv_files_rec('dataframes')
all_csv_files

['dataframes/conjunction/però.csv',
 'dataframes/conjunction/appena.csv',
 'dataframes/conjunction/perché.csv',
 'dataframes/conjunction/e.csv',
 'dataframes/conjunction/quasi.csv',
 'dataframes/conjunction/siccome.csv',
 'dataframes/conjunction/sia.csv',
 'dataframes/conjunction/quanto.csv',
 'dataframes/conjunction/come.csv',
 'dataframes/conjunction/mentre.csv',
 'dataframes/conjunction/comunque.csv',
 'dataframes/conjunction/se.csv',
 'dataframes/conjunction/né.csv',
 'dataframes/conjunction/oppure.csv',
 'dataframes/conjunction/quando.csv',
 'dataframes/conjunction/che.csv',
 'dataframes/conjunction/ma.csv',
 'dataframes/conjunction/o.csv',
 'dataframes/conjunction/per cui.csv',
 'dataframes/conjunction/così.csv',
 'dataframes/noun/sapere.csv',
 'dataframes/noun/chilometro.csv',
 'dataframes/noun/pizza.csv',
 'dataframes/noun/completo.csv',
 'dataframes/noun/bagno.csv',
 'dataframes/noun/albergo.csv',
 'dataframes/noun/basso.csv',
 'dataframes/noun/perché.csv',
 'dataframes/noun/p

In [64]:
all_df = pd.DataFrame()
for path in all_csv_files:
    assert(os.path.exists(path))
    # print(path)
    df = pd.read_csv(path)
    all_df = pd.concat([all_df, df], ignore_index=True)
len(all_df), all_df.columns

(2433,
 Index(['term_italian', 'is_base', 'base_lemma_italian', 'base_lemma_english',
        'part_of_speech', 'translation_english', 'topics', 'subtype',
        'person_number', 'gender', 'plurality', 'mood', 'tense',
        'is_comparative', 'is_superlative', 'is_compound', 'article_type',
        'article_italian', 'added_particle_italian', 'preposition', 'notes',
        'example_sentence_italian', 'example_sentence_english'],
       dtype='object'))

In [65]:
all_df['base'] = all_df.apply(lambda row: f"{row['base_lemma_italian']} - {row['base_lemma_english']}", axis=1)
all_df['base'].value_counts()

base
subire - to undergo                              48
dovere - obligation; must/have to                41
essere - to be                                   36
arrivare - to arrive                             31
accorgersi - to notice/realize                   30
                                                 ..
ognuno - everyone (masculine or mixed gender)     1
ognuno - everyone (feminine)                      1
ognuno - each one (masculine)                     1
ognuno - each one (feminine)                      1
tra - between/among                               1
Name: count, Length: 203, dtype: int64

In [ ]:
all_df[all_df['base'] == 'ognuno - everyone (feminine)']

,term_italian,is_base,base_lemma_italian,base_lemma_english,part_of_speech,translation_english,topics,subtype,person_number,gender,...,is_superlative,is_compound,article_type,article_italian,added_particle_italian,preposition,notes,example_sentence_italian,example_sentence_english,base


In [49]:
harry_potter_words = set(text.lower().replace('\n', ' ').replace('.', ' ').replace(',', ' ').replace('!', ' ').replace('?', ' ').split(' '))
# ignore empty strings
harry_potter_words.discard('')
# ignore numbers
harry_potter_words = {w for w in harry_potter_words if not w.isdigit()}
len(harry_potter_words)

215

In [50]:
IMPORTANT_COLS = ['term_italian', 'part_of_speech', 'translation_english', 'base_lemma_italian', 'base_lemma_english']

In [51]:
all_df[all_df['term_italian'].isin(harry_potter_words)][IMPORTANT_COLS]

,term_italian,part_of_speech,translation_english,base_lemma_italian,base_lemma_english
1,appena,conjunction,as soon as,appena,just/barely; as soon as
3,e,conjunction,and,e,and
9,come,conjunction,as/like,come,as/like/how
10,come,conjunction,as,come,as/like/how
12,se,conjunction,if/whether,se,if
...,...,...,...,...,...
2334,come,preposition,as,come,as/like/how
2338,sulla,preposition,on the,su,on
2345,in,preposition,in/into,in,in/into
2346,nel,preposition,in the,in,in/into


In [52]:
missing_words = harry_potter_words - set(all_df['term_italian'])
print(len(missing_words))
missing_words

124


{'bryce',
 'chiaramente',
 'chilometri',
 "cinquant'anni",
 'collina',
 'colpo',
 'cominciava',
 'completo',
 'comunque',
 'convenivano',
 'corsa',
 'cresceva',
 'crogiolò',
 'cui',
 'cuoca',
 'curiosità',
 "d'estate",
 "d'oro",
 "dell'assassino:",
 'desolata',
 'dimora',
 'disabitata',
 'discutere',
 'dominava',
 'eccitazione',
 'elegante',
 'entrata',
 'facciata',
 'fece',
 'figlio',
 'finestre',
 'fingersi',
 'fino',
 'fosse',
 'frank',
 'freddi',
 'fronzoli',
 'fu',
 'ghiaccio',
 'giornata',
 'giù',
 'grandioso',
 'hangleton',
 'hangletoniani',
 'imponente',
 'impopolari',
 'importava',
 'improvvisamente',
 'inchiodate',
 'incolta',
 'ingresso',
 "l'edera",
 "l'edificio",
 "l'identità",
 "l'impiccato",
 'little',
 'locale',
 'là',
 'malcelata',
 'mancavano',
 'marito',
 'mezzo',
 'modo:',
 'moglie',
 'morivano',
 'morti',
 'naturali',
 'non',
 'notte',
 'occhi',
 'omicidi',
 'ora',
 'ormai',
 'passati',
 'persone',
 'pettegolezzi',
 'più',
 'polizia',
 'pub',
 'qualcosa',
 "quell'u